# Count of Oil and Gas Wells in the US by County

## Explanation of Project

While working as an intern for an environmental economics organization, one of the projects I was a part of was a rangeland/cropland conservation project throughout several US states (Texas, Wyoming, New Mexico, Oklahoma, Colorado, Kansas, and Nebraska). The level of granularity for the project was at the county level, and we needed a way to differentiate the counties based on their oil and natural gas output.

The first way I did this was by finding production amounts by county for oil and natural gas, which were readily available online. After completing this, I was also asked to verify/support this approach by also figuring out the total number of oil and natural gas wells per county.

While this projet has already been completed and visualized and completed on the first website below, the data from that website is raw and only labels the wells by Latitude and Longitude, without their corresponding county name. Because of that, my approach to labeling the wells with their county name and aggregating that is layed out below. The total number of wells was 1.7 million, which, as shown below, put a constraint on the realistic methods I could use to assign the correct county names to each latitude/longitude

## Data Sources

I accessed the well data from this website: https://www.fractracker.org/2015/08/1-7-million-wells/
It has a listing of all the wells in the United States, excluding offshore, North Carolina, and Texas

To get the data for Texas, I went to this site: http://www.rrc.state.tx.us/oil-gas/research-and-statistics/well-information/well-distribution-by-county-well-counts/

The well data for Texas was already aggregated by county, so it is not included as part of assigning the county names to the latitudes and longitudes.

## Process to Aggregate the Data

It took a little bit of trial and error to find the best way to get the county information from the Lat/Longs

I found a python library that could do it with something called Noominatim from open street map, but it had a request limit that was prohibitive for the amount of entries I had. The process of using Noominatim to query the GEOID information is not included in this notebook, but it is esentially the same as querying the US Census website

I then found the Census API where you can use a website link with specific Lat and Longs inputted to return a JSON of descriptive data about the location, including county name and GEOID. This method was faster, but still too slow to process the entire dataset.

So my next step was to download a dataset of Latitude/Longitude pairs with associated county information from the US Census into a text file. These Lat and Long pairs that had GEOID labels could be used to to train a K-Nearest Neighbors algorithm to predict the remainder of the un-labeled Lat/Longs. I chose K-Nearest Neighbors as opposed to another algorithm because the input feature set was two dimensional, and the problem itself was inherently spatial, so I thought it would be a good fit.


*As a note, the part of this project that is not included in this notebook is the final solution I used to get the county names. I used QGIS along with a shapefile which had polygon boundaries of each county, so that a specific Lat/Long point could be place within a certain county. This is important, as the nearest neighbor algorithm could get it wrong if there is not a high enough density of points along the border between two counties, and the closest point is within the incorrect county*

## Methodolgy and High-Level Explanation of Notebook

While the final method used in QGIS to get the GEOID information from the Lat/Longs was used instead of the K-Nearest Neighbors algorithm, the notebook below shows my process. After first setting aside a testing set to test the KNN algorithm on, and seeing where it failed, I was interested to see how many new points it would take (by querying the census API) to get to a certain acceptable level of accuracy with the KNN algorithm. I created some functions to do this (as well as using the census API to generate testing data), and after a couple of iterations, realized it was prohibitively slow.

*With that, the file I save below called 'GEOIDS_Progress.csv' is just a checkpoint so that I can shutdown the notebook and keep the extra labeled Lat/Longs that I had already queried from the census API and not have to start over with redefining the original versions of each of the dataframes.*

*The file: 'wells_original.csv' is just an aggregation of the 3 raw data files taken from the website, trimmed down to just the states I was looking at for this project.*

*The file: 'Original_Census_Data.csv' is the original text file with Lat/Longs and GEOID information taken from the census website, trimmed down to just the states in this project.*

*The file: 'Data_for_GIS.csv' is the well data trimmmed down to just the Lat/Longs, the state code, and the state abbreviation (with duplicates removed), to be fed into QGIS along with the county shapefile.*

*The file: 'predicted.csv' has the predicted county ID's from the KNN model for each Lat/Long combination.*

## Next Step: Importing Libraries and Cleaning the Data

In [1]:
#######========> Import libraries needed for this notebook

import pandas as pd #Data manipulation library
import numpy as np #Numerical python
import requests #Libary to get the information in JSON format from the census API
import datetime #Library I used to time how long a chunk of code takes to run (just for curiosity)
from sklearn.metrics import accuracy_score #Metric to test accuracy of KNN
from sklearn.neighbors import KNeighborsClassifier #KNN algorithm

#######========> End importing of libraries needed for this notebook

#######========> Define the columns of each file to be used, and data types

#The census link to access information about a Lat,Long pair
site = 'http://data.fcc.gov/api/block/find?format=json'

cols = ['State','Well_Name','Operator','Type','Status',
        'Spud Date','Longitude','Latitude','API']

types = {'State': str, 'Well_Name': str, 'Operator': str, 
         'Type': str, 'Status': str, 'Spud Date': str, 
         'Longitude': str, 'Latitude': str, 'API': str}

#######========> End column type definition

#######========> Read the three files containing well data into dataframes

#Define file path for the well data
path = '~/Desktop/Earth Economics/AngelaWork/OilandGas/'

data1 = pd.read_csv(path + 'Well_Data/wells1.csv', 
                    encoding="ISO-8859-1", 
                    usecols = cols, 
                    dtype = types)
data2 = pd.read_csv(path + 'Well_Data/wells2.csv', 
                    encoding="ISO-8859-1", 
                    usecols = cols, 
                    dtype = types)
data3 = pd.read_csv(path + 'Well_Data/wells3.csv', 
                    encoding="ISO-8859-1", 
                    usecols = cols, 
                    dtype = types)

#######========> End dataframe creation from original csv files

**Now, concatenate the three dataframes into one, and save this file as a csv**

In [2]:
#######========> Define list of states in the project, and the mapping for their codes

#List the states that I am interested in to trim down the file
states = ['WY','NM','OK','CO','KS','NE']

#Dictionary mapping the states to their codes
state_dict = {'CO': '08', 'KS': '20', 'NE': '31',
              'NM': '35', 'OK': '40', 'WY': '56'}

#######========> End state abbreviation/code definition

#######========> Concatenate dataframes, trim down to states in list, and add relevant columns

#Concatenate the three original files into one.
wells_og = pd.concat([data1, data2, data3])

#Trim down the file to only include the states relevant to the project
wells_og = wells_og[wells_og.State.isin(states)]

#Combined column to get a shorter list of unique Lat/Longs
wells_og['LatLong'] = wells_og.Latitude + ',' + wells_og.Longitude

#Column of the state codes for each state
wells_og['State_Code'] = wells_og.State.map(state_dict)

#######========> End dataframe concatenation and preparation

#######========> Save wells data to csv, and display the prepared dataframe

#Save the original wells data file
wells_og.to_csv(path + 'Well_Data/wells_original.csv')

#Display the first five rows of the data
wells_og.head()

#######========> End file saving and display

,State,Well_Name,Operator,Type,Status,Spud Date,Longitude,Latitude,API,LatLong,State_Code
154331,WY,JOHN F JESSEN 1,Z & S CONSTRUCTION CO INC,Oil Well,Producing Oil Well,19850727,-104.05490,41.30750,2120376,"41.30750,-104.05490",56
154332,WY,JESSEN 24-26,BEAR OIL AND GAS INC,Oil Well,Subsequent Report of Abandonment,20120917,-104.05519,41.31780,2120969,"41.31780,-104.05519",56
154333,WY,MALM 42-34,RANCH OIL COMPANY,Oil Well,Producing Oil Well,20100814,-104.05672,41.48507,2120659,"41.48507,-104.05672",56
154334,WY,AFTON GREEN 1,HARRISON SAM G,Oil Well,Subsequent Report of Abandonment,19641223,-104.05738,41.62074,1505009,"41.62074,-104.05738",56
154335,WY,CHARLES 5-10 1,DARRAH JOHN JAY JR,Oil Well,Subsequent Report of Abandonment,19961207,-104.06029,43.02864,2720972,"43.02864,-104.06029",56


**The file 'US.txt' is the downloaded file from the US Census website, it is a tab separated file, so can be read into pandas with the standard read_csv function, just with the separator changed to '\t' instead of ','**

**The code below loads the file in, and does similar trimming and column addition to the wells dataframes above**

In [3]:
#######========> Read US Census datafile into pandas dataframe, trim, and clean data

#Use tab separator, and define columns to use and their data types
file = pd.read_csv(path + 'Well_Data/US.txt', sep = '\t', usecols = [4,5,10,11], 
                   header = None, dtype = {4: str, 5:str, 10:str, 11:str})

#Name the columms appropriately
file.columns = ['Latitude','Longitude','State','County']

#There were some Lats/Longs that didn't have a county so I dropped those
file = file[file.County.notnull()] 

#Limit the file to just the states related to this project
file = file[file.State.isin(states)]

#Create a column to match to the wells dataframe
file['LatLong'] = file.Latitude + ',' + file.Longitude

#Map the states to their state codes
file['State_Code'] = file.State.map(state_dict)

#Create a GEOID by concatenating the state codes with the county #'s
file['GEOID'] = file['State_Code'] + file.County

#Get rid of any duplicate Lat/Long combinations to trim the dataset
file = file.drop_duplicates('LatLong')

#Set index as the latitudes and longitudes concatenated together
file = file.set_index('LatLong')

#Reorder the census columns
file = file[['Latitude','Longitude','State','State_Code','County','GEOID']]

#######========> End dataframe creation, trimming, and cleaning

#######========> Save trimmed data to csv file and display the dataframe

#Save the original Census data to csv
file.to_csv(path + 'Well_Data/Original_Census_Data.csv')

#Display the first five rows of the Census data
file.head()

#######========> End file saving and display

,Latitude,Longitude,State,State_Code,County,GEOID
LatLong,,,,,,
"38.27252,-99.93346",38.27252,-99.93346,KS,20,135,20135
"38.20001,-97.45587",38.20001,-97.45587,KS,20,113,20113
"39.19027,-96.5625",39.19027,-96.5625,KS,20,161,20161
"37.15252,-98.5498",37.15252,-98.5498,KS,20,007,20007
"38.47823,-98.42495",38.47823,-98.42495,KS,20,159,20159


**My next step is to trim down the original wells data set into a version that will be used in QGIS. This involves leaving just the columns needed to get the county information. These are: Lat/Longs, States, State_Codes**

In [4]:
#######========> Create new wells dataframe which gets rid of columns not needed in QGIS

#Drop duplicates and create a new wells frame to be manipulated
wells = wells_og.drop_duplicates('LatLong')

#Set the index to the concatenated Lat/Longs
wells = wells.set_index('LatLong')

#Define the columns to be dropped
drop_cols = ['Operator','Well_Name','Type','Status','Spud Date','API']

#Get rid of unecessary columns
wells = wells.drop(drop_cols, axis = 1)

#Reorder the columns
wells = wells[['Latitude','Longitude','State','State_Code']]

#######========> End QGIS ready dataframe creation

#######========> Save the dataframe to a csv file and display it

#Create a file to input into the GIS software
wells.to_csv(path + 'Predicted_Counties_For_LatLong/Data_for_GIS.csv')

#Display first five rows of trimmed wells data
wells.head()

#######========> End file saving and display

,Latitude,Longitude,State,State_Code
LatLong,,,,
"41.30750,-104.05490",41.30750,-104.05490,WY,56
"41.31780,-104.05519",41.31780,-104.05519,WY,56
"41.48507,-104.05672",41.48507,-104.05672,WY,56
"41.62074,-104.05738",41.62074,-104.05738,WY,56
"43.02864,-104.06029",43.02864,-104.06029,WY,56


**Now I want to combine the files in order to more easily use the nearest neighbors and generate new data when needed. So the join used is a full outer join, so that all unique Lat/Long combinations are included, and if there is any exact matches in the Census dataset, I won't need to re-predict or search those.**

In [5]:
#Merge the census data and the trimmed wells data with an outer join
combined = file.merge(wells, how = 'outer')

#Display first five rows of the combined data set. Many GEOIDS are null
combined.head()

,Latitude,Longitude,State,State_Code,County,GEOID
0,38.27252,-99.93346,KS,20,135,20135
1,38.20001,-97.45587,KS,20,113,20113
2,39.19027,-96.5625,KS,20,161,20161
3,37.15252,-98.5498,KS,20,007,20007
4,38.47823,-98.42495,KS,20,159,20159


## Checkpoint for Current Progress from Census API

**So what I did, just out of curiosity's sake, is build a function to generate new random samples to see how much the addition of a few hundred to a few thousand labeled Lat/Longs would improve the algorithm (which it definitely did).**

**This cell below is a checkpoint, as when I wanted to close this notebook, or it timed out or I had to force quit it for any reason, I wouldn't lose all of the new GEOID's I had gathered. I saved a file called 'GEOID_Progess.csv' so that I could just start from this cell when I restarted the notebook**

In [11]:
#######========> Import libraries needed for this notebook and define Census API link

import pandas as pd #Data manipulation library
import numpy as np #Numerical python
import requests #Libary to get the information in JSON format from the census API
import datetime #Library I used to time how long a chunk of code takes to run (just for curiosity)
from sklearn.metrics import accuracy_score #Metric to test accuracy of KNN
from sklearn.neighbors import KNeighborsClassifier #KNN algorithm

#The census link to access information about a Lat,Long pair
site = 'http://data.fcc.gov/api/block/find?format=json'

#######========> End importing of libraries needed for this notebook

#######========> Bring in the progress file, define column data types, and display

#Define all the columns as having data type of string
dtypes = {'Latitude' : str, 'Longitude' : str, 'State' : str, 
          'State_Code' : str, 'County' : str, 'GEOID' : str}

#Read the progress file into pandas dataframe
combined = pd.read_csv(path + 'Predicted_Counties_For_LatLong/GEOIDS_Progress.csv',
                       dtype = dtypes)

#Display first five rows of the dataframe
combined.head()

#######========> End display and reading of csv file

,Latitude,Longitude,State,State_Code,County,GEOID
0,38.27252,-99.93346,KS,20,135,20135
1,38.20001,-97.45587,KS,20,113,20113
2,39.19027,-96.5625,KS,20,161,20161
3,37.15252,-98.5498,KS,20,007,20007
4,38.47823,-98.42495,KS,20,159,20159


## Define Several Functions for Sample Generation and Model Training/Prediction

**The gen_new algorithm just takes a list of the states and the number of samples I want to generate, so I can choose the subset of states and increase or decrease the amount of new data I want to generate (takes a long time).**

In [12]:
#######========> Define function to generate new correct Lat/Longs paired with GEOID's for training

def gen_new(states, samples):
    
    #iterate over the states given to the function
    for state in states:
        
        #Define the sample of the dataframe for the indexes to be pulled from
        df_trim = combined[(combined.GEOID.isnull())&(combined.State == state)]
        
        #Define if statement to ensure more samples are asked for than exist
        if df_trim.shape[0] < samples:
            df = df_trim.sample(df_trim.shape[0])
        else:
            df = df_trim.sample(samples)
        
        #Iterate over the indexes in the sampled dataframe
        for index in df.index:
            
            #Definte the latitude
            Lat = combined.loc[index, 'Latitude']
            
            #Define the longitude
            Long = combined.loc[index, 'Longitude']
            
            #Get the county information from the Census API
            geoid = requests.get(site + '&latitude=' + Lat + 
                                 '&longitude=' + Long + 
                                 '&showall=true').json()['County']['FIPS']
            
            #Assign the geoid give by the census to that row
            combined.loc[index, 'GEOID'] = geoid
            
        #Get confirmation that it is runnning properly
        print(state + ' is finished...')
    return None

#######========> End function for creating new training examples

**So in order to accurately see how well the nearest neighbors algorithm was performing, I needed to create an out of sample testing set. So this takes a set and a number of samples, and returns a dataframe that can be used to accurately test the accuracy of the prediction.**

**I could have split the data into training and testing samples and done cross validation to have a larger testing set for determining the accuracy score which wouldn't have taken as long as developing new samples, but I just leaned more towards having the entire data for training. Also, since I was able to generate truly out of sample data by not adding these generated samples to the data, I thought that would be a plus too.**

In [13]:
#######========> Define function to generate an out of sample testing set

def test_sample(state, sample_num):
    
    #Create empty lists to store the values
    Lats, Longs, geoids = [], [], []
    
    #Define the trimmed dataframe that test sample will be pulled from
    df_trim = combined[(combined.GEOID.isnull())&(combined.State == state)]
    
    #Define if statement to ensure more samples are asked for than exist
    if df_trim.shape[0] < sample_num:
        df = df_trim.sample(df_trim.shape[0])
    else:
        df = df_trim.sample(sample_num)
    
    #Loop over the indexes in the trimmed dataframe
    for index in df.index:
        
        #Store the latitudes and Longitudes at the index
        Lat = combined.loc[index, 'Latitude']
        Long = combined.loc[index, 'Longitude']
        
        #Take the Lat/Longs for this iteration and get the county info
        geoid = requests.get(site+'&latitude='+Lat+'&longitude=' 
                            +Long+'&showall=true').json()['County']['FIPS']
        
        #Append the Lat/Long/Geoid to each of their respective lists
        Lats.append(Lat)
        Longs.append(Long)
        geoids.append(geoid)
        
    #Create a numpy array of all three lists (Lat, Long, Geoids)
    LatLong = np.array([Lats, Longs, geoids])
    
    #Turn this into a dataframe which gets returned from the function
    test_frame = pd.DataFrame(LatLong.T,columns=['Latitude','Longitude','GEOID'])
    
    return test_frame

#######========> End function to define out of sample testing set

**This function (train_algo) takes the list of states, the number of trials I want to run, and the number of samples per trial and returns the accuracy scores for each state (100% being the best).**

**My reasoning for doing the algorithm at the state level was that this would eliminate any confusion when two states border each other and there isn't enough data for the algorithm to determine which state it is in. This doesn't solve the problem of data being right on the county border, but it makes it a little more accurate at least.**

**When testing the algorithm, I tested many different values for n_neighbors (how many nearest points the algorithm looks at), and setting it to 1 always produced the best accuracy that I found.**

**I did try other algorithms as well such as Random Forest and Linear Regression, but nearest neighbors worked better**

In [14]:
#######========> Define function to train the data on the KNN model

def train_algo(states, numtrials, sample_num):
    
    #Define the X and Y columns to be used for training the model
    xcols = ['Latitude','Longitude']
    ycols = ['GEOID']
    
    #Create an empty list to store the accuracy values for each state
    accuracy = []
    
    #Define the KNN model with N_Neighnors = 1
    model = KNeighborsClassifier(n_neighbors = 1)
    
    #Iterate over each of the states passed to the function
    for state in states:
        
        #Declare a current value variable
        current = 0
        
        #Define the dataframe for the given state
        df = combined[(combined.GEOID.notnull())&(combined.State == state)]
        
        #Iterate over the number of trials passed to the function
        for trial in range(numtrials):
            
            #Define the x_train and y_train sets based on the x and y columns
            x_train = df[xcols]
            y_train = df[ycols]
            
            #Call the test_sample function to create a new dataframe of testing data
            test_frame = test_sample(state, sample_num)
            
            #Define the testing set based on the function that gets new testing samples
            x_test, y_test = test_frame[xcols], test_frame[ycols].values.ravel()
            
            #Fit the model to the x_train and y_train dataframes
            model.fit(x_train,y_train.values.ravel())
            
            #Use the trained model to make a prediction on the testing set
            y_pred = model.predict(x_test)
            
            #Calcualte the accuracy score of y_compared to y_predicted
            current += accuracy_score(y_test, y_pred)
        
        #Append the average of the accuracy scores of the num_trials to the accuracy list
        accuracy.append(current/numtrials)
        
        #Use this print statement to give user feedback that code is functioning properly
        print(state + ' is finished...')
        
    #Return the list with the average accuracy scores calculated
    return accuracy

#######========> End function creation to train the KNN model

**This chunk of code below just generates new data by calling the Census API for each state based on what 'samples' is set to**

In [15]:
#######========> Implement code to generate new data points, and calculate time elapsed

#Define starting time
a = datetime.datetime.now().replace(microsecond = 0)

#Define set of states to be passed to gen_new function
states = ['WY','NM','OK','CO','KS','NE']

#Define number of new data points to be queried from the Census API
samples = 10

#Call the function with the above parameters, it returns none, just overwrites existing dataframe
gen_new(states, samples)

#Define ending time and print the difference between the two
b = datetime.datetime.now().replace(microsecond = 0)
print(b-a)

#######========> End generation of new data points

WY is finished...
NM is finished...
OK is finished...
CO is finished...
KS is finished...
NE is finished...
0:00:36


**This chunk of code below generates testing data and calculates the accuracy score for each state**

In [16]:
#######========> Implement code to train the KNN model, and calculate time elapsed

#Define starting time
a = datetime.datetime.now().replace(microsecond = 0)

#Define set of states to be passed to train_algo function
states = ['WY','NM','OK','CO','KS','NE']

#Define number of test samples to be generated by test_sample function
sample_num = 25

#Define the number of trials for the training function to average over
numtrials = 5

#train_algo function returns the list of average accuracy scores
accuracy = train_algo(states, numtrials, sample_num)

#Define the ending time and print the difference
b = datetime.datetime.now().replace(microsecond = 0)
print(b-a)

#######========> End code to train the KNN model

WY is finished...
NM is finished...
OK is finished...
CO is finished...
KS is finished...
NE is finished...
0:05:22


**Below, the current accuracy scores by state fed into the training function are printed**

In [28]:
for i in range(len(states)):
    print('For state: ' + str(states[i]) + ', accuracy is: ' + str(round(accuracy[i],2)) + '\n')
print('The mean accuracy score is: ' + str(round(np.mean(accuracy),2)))

For state: WY, accuracy is: 0.94

For state: NM, accuracy is: 0.98

For state: OK, accuracy is: 0.95

For state: CO, accuracy is: 0.98

For state: KS, accuracy is: 0.93

For state: NE, accuracy is: 0.94

The mean accuracy score is: 0.95


In [29]:
#Save the current progress of new data point generation to the progress file
combined.to_csv(path + 'Predicted_Counties_For_LatLong/GEOIDS_Progress.csv',
                index = False)

## Use the KNN Model to Predict Values and Write to File

**Once I was satisfied with the predictive accuracy of the algorithm, I filled in the null GEOIDS with the KNN prediction, using the below code**

In [31]:
#######========> Implement code to fill the null GEOID values with the KNN predicted values

#Define the X and Y columns passed into the KNN model
xcols = ['Latitude','Longitude']
ycols = ['GEOID']

#Define the states to be passed into the KNN model
states = ['WY','NM','OK','CO','KS','NE']

#Define the KNN model with 1 neighbor
model = KNeighborsClassifier(n_neighbors = 1)

#Iterate over the desired states
for state in states:
    
    #Define dataframes for the given state
    df_train = combined[(combined.GEOID.notnull())&(combined.State == state)]
    df_test = combined[(combined.GEOID.isnull())&(combined.State == state)]
    
    #Passing over the state if it doesn't have any null values to predict
    if df_train[xcols].shape[0] > 0:
        
        #Define x_train, y_train, and the x values to be predicted
        x_train = df_train[xcols]
        y_train = df_train[ycols]
        x_test  = df_test[xcols]
        
        #Fit the model to the training values
        model.fit(x_train,y_train.values.ravel())
        
        #Write the model predictions over the GEOID null column values
        combined.loc[(combined.GEOID.isnull())&
                     (combined.State == state),'GEOID'] = model.predict(x_test)
    
    #If the state doesn't have any null values, just pass on move on to the next
    else:
        pass

#Add a new Lat/Long combo columns
combined['LatLong'] = combined.Latitude + ',' + combined.Longitude
    
#Use this to ensure that all null values for GEOID were predicted with the KNN model
combined.isnull().sum()

#######========> End code to fill null GEOID values with KNN predicted GEOIDs

Latitude           0
Longitude          0
State              0
State_Code         0
County        603337
GEOID              0
LatLong            0
dtype: int64

**Merge the original wells dataframe with the newly filled in dataframe that has Lat/Longs and GEOIDS. Doing a left join so that only the Lat/Long combinations that are in the original file are filled in**

In [32]:
#Merge the original wells frame with the newly filled combined frame
with_id = pd.merge(wells_og, combined[['LatLong','GEOID']],
                   how = 'left',
                   on = 'LatLong')

#Display the first five rows of the original wells dataset with the GEOIDS included
with_id.head()

,State,Well_Name,Operator,Type,Status,Spud Date,Longitude,Latitude,API,LatLong,State_Code,GEOID
0,WY,JOHN F JESSEN 1,Z & S CONSTRUCTION CO INC,Oil Well,Producing Oil Well,19850727,-104.05490,41.30750,2120376,"41.30750,-104.05490",56,56021
1,WY,JESSEN 24-26,BEAR OIL AND GAS INC,Oil Well,Subsequent Report of Abandonment,20120917,-104.05519,41.31780,2120969,"41.31780,-104.05519",56,56021
2,WY,MALM 42-34,RANCH OIL COMPANY,Oil Well,Producing Oil Well,20100814,-104.05672,41.48507,2120659,"41.48507,-104.05672",56,56021
3,WY,AFTON GREEN 1,HARRISON SAM G,Oil Well,Subsequent Report of Abandonment,19641223,-104.05738,41.62074,1505009,"41.62074,-104.05738",56,56015
4,WY,CHARLES 5-10 1,DARRAH JOHN JAY JR,Oil Well,Subsequent Report of Abandonment,19961207,-104.06029,43.02864,2720972,"43.02864,-104.06029",56,56027


**Lastly we will group the new dataframe by GEOID, aggregating by count so that the count of wells by county can be easily transferred to where the project data is held in excel**

In [33]:
#Group by the GEOIDs and aggregate with a count
predicted = with_id.groupby('GEOID')['GEOID'].count()

#Save these final counts to a csv to be merged where the project was originally held in excel
predicted.to_csv(path + 'Predicted_Counties_For_LatLong/KNN_predicted.csv',
                 header = True)

#Display the first 5 rows of the predicted groupby aggregation
predicted.head()

GEOID
08001    1737
08003       1
08005     339
08007     215
08009     309
Name: GEOID, dtype: int64